In [18]:
%load_ext dotenv
%dotenv

from config import *

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [17]:
from config import *

In [19]:
def load_geolocation_data(json: str) -> pd.DataFrame:
    data = pd.read_json(json)
    cleaned_data = pd.DataFrame()

    cleaned_data['id'] = data['id']
    cleaned_data['country'] = data['country']
    cleaned_data['region'] = data.apply(lambda row: row['country'] if pd.isnull(row['region']) else row['region'], axis=1)
    cleaned_data['location'] = data['location']
    cleaned_data['timezone'] = data['timezone']

    normalize_loc = pd.json_normalize(cleaned_data['location'])

    cleaned_data[[col for col in normalize_loc.columns]] = normalize_loc
    cleaned_data.drop(columns=['location'], inplace=True, axis=1)

    return cleaned_data

In [20]:
data = load_geolocation_data('full.json')
table_name = 'geolocations'
engine = create_engine(connection_string)
data.to_sql(table_name, engine, if_exists='replace', index=False)
engine.dispose()